
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [2]:
 !pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [4]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y4", label="solidos")

/tmp/ipykernel_1357434/178310800.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 18:48:23.893989: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 18:48:23.955509: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 18:48:23.955544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 18:48:23.956585: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 18:48:23.962295: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 18:48:23.963348: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [7]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [8]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [9]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[70,35]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[70, 35], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  7.90505901799952
Stopped at epoch:  126
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.3354431810003007
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.289271448999898
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.32729775000007066
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.2512864919999629
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.33568530100001226
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 7 ++++++++

2024-03-24 18:49:23.269822: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.38061428800028807
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_2] | 24 ++++++++++++++++++
Elapsed time:  0.3094636010000613
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_2] | 25 ++++++++++++++++++
Elapsed time:  0.3417068159997143
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_3] | 1 ++++++++++++++++++
Elapsed time:  0.47949315499954537
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_3] | 2 ++++++++++++++++++
Elapsed time:  0.7199607279999327
Stopped at epoch:  13
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_3] | 3 ++++++++++++++++++
Elapsed time:  0.4461622190001435
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_3] | 4 ++++++++++++++++++
Elapsed time:  0.4204493659999571
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step


,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_6_0,0.800784,0.323958,0.144426,0.290933,24.025564,73.903984,26.830833,51.751869
model_1_6_1,0.802168,0.320214,0.122729,0.283578,23.858652,74.313354,27.511248,52.288727
model_1_6_2,0.803522,0.316424,0.103386,0.276660,23.695312,74.727585,28.117840,52.793636


DataFrame salvo em ../results/metrics_1_6
+++++++++++ [1_7] | 1 ++++++++++++++++++
Elapsed time:  0.32351605300027586
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_7] | 2 ++++++++++++++++++
Elapsed time:  0.38508833200012305
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_7] | 3 ++++++++++++++++++
Elapsed time:  0.27036208100071235
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [1_7] | 4 ++++++++++++++++++
Elapsed time:  0.371718589000011
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_7] | 5 ++++++++++++++++++
Elapsed time:  0.34107737300018925
Stopped at epoch:  6
1/1 [==============================] - 0s 27ms/step
+++++++++++ [1_7] | 6 ++++++++++++++++++
Elapsed time:  0.302769038000406
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_7] | 7 ++++++++++++++++++
Elapsed time:  0.2823308019

...

DataFrame salvo em ../results/metrics_1_7
+++++++++++ [1_8] | 1 ++++++++++++++++++
Elapsed time:  0.34846637200007535
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_8] | 2 ++++++++++++++++++
Elapsed time:  0.32854082400081097
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_8] | 3 ++++++++++++++++++
Elapsed time:  0.2890592089997881
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_8] | 4 ++++++++++++++++++
Elapsed time:  0.2618195300001389
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_8] | 5 ++++++++++++++++++
Elapsed time:  0.3935692079994624
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_8] | 6 ++++++++++++++++++
Elapsed time:  0.24381050600004528
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_8] | 7 ++++++++++++++++++
Elapsed time:  0.305556910

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_8_0,0.807757,0.430415,0.722696,0.589180,23.184504,79.513069,9.179605,46.415100
model_1_8_1,0.827209,0.521135,0.707525,0.646435,20.838673,66.848640,9.681799,39.946342
model_1_8_2,0.844192,0.599872,0.691395,0.695714,18.790537,55.857140,10.215771,34.378750
model_1_8_3,0.858408,0.664783,0.674466,0.735837,17.075991,46.795700,10.776165,29.845486
model_1_8_4,0.869997,0.716489,0.656672,0.767208,15.678434,39.577599,11.365201,26.301182
model_1_8_5,0.879305,0.756789,0.638109,0.791008,14.555910,33.951805,11.979687,23.612190
model_1_8_6,0.886847,0.788467,0.618239,0.808993,13.646279,29.529642,12.637451,21.580297
model_1_8_7,0.892892,0.812858,0.597513,0.822089,12.917249,26.124689,13.323532,20.100624
model_1_8_8,0.897804,0.832032,0.575162,0.831551,12.324819,23.448042,14.063419,19.031652
model_1_8_9,0.901781,0.846939,0.551590,0.838050,11.845270,21.367075,14.843731,18.297300


DataFrame salvo em ../results/metrics_1_8
+++++++++++ [1_9] | 1 ++++++++++++++++++
Elapsed time:  0.7073503010005879
Stopped at epoch:  9
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_9] | 2 ++++++++++++++++++
Elapsed time:  0.3016809390001072
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_9] | 3 ++++++++++++++++++
Elapsed time:  0.33174901400070667
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_9] | 4 ++++++++++++++++++
Elapsed time:  0.32709385999987717
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_9] | 5 ++++++++++++++++++
Elapsed time:  0.31156397300037497
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_9] | 6 ++++++++++++++++++
Elapsed time:  0.265439380999851
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_9] | 7 ++++++++++++++++++
Elapsed time:  0.3499657779

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_9_24,0.822117,0.553997,-0.585060,0.141895,21.452772,42.639656,96.648010,68.055115
model_1_9_23,0.829467,0.563427,-0.497606,0.179546,20.566376,41.738113,91.315552,65.069008
model_1_9_22,0.830601,0.564075,-0.454801,0.195448,20.429548,41.676170,88.705551,63.807877
model_1_9_21,0.833808,0.571830,-0.414525,0.214969,20.042780,40.934738,86.249771,62.259724
model_1_9_16,0.834125,0.563283,-0.319193,0.244007,20.004583,41.751862,80.436920,59.956764
model_1_9_15,0.834809,0.563742,-0.301700,0.250630,19.922075,41.707996,79.370323,59.431473
model_1_9_20,0.838118,0.580878,-0.365300,0.238556,19.522989,40.069717,83.248306,60.389072
model_1_9_19,0.839783,0.584909,-0.343577,0.248989,19.322216,39.684372,81.923729,59.561642
model_1_9_18,0.842335,0.591318,-0.313581,0.263925,19.014498,39.071598,80.094742,58.377068
model_1_9_17,0.844515,0.597021,-0.287170,0.277120,18.751596,38.526443,78.484344,57.330585


DataFrame salvo em ../results/metrics_1_9
Testando combinacao2: Hidden Size=[70, 35], regularizer=0.02, learning_rate=0.1
+++++++++++ [2_0] | 1 ++++++++++++++++++
Elapsed time:  0.49327372799962177
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_0] | 2 ++++++++++++++++++
Elapsed time:  0.056886888000008184
Stopped at epoch:  0
1/1 [==============================] - 0s 22ms/step
+++++++++++ [2_0] | 3 ++++++++++++++++++
Elapsed time:  0.05631888700008858
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_0] | 4 ++++++++++++++++++
Elapsed time:  0.034845567000047595
Stopped at epoch:  0
1/1 [==============================] - 0s 21ms/step
+++++++++++ [2_0] | 5 ++++++++++++++++++
Elapsed time:  0.04554053100036981
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_0] | 6 ++++++++++++++++++
Elapsed time:  0.04668387299989263
Stopped at epoch:  0
1/1 [=============================